### 1.6: Intro to NLP and Network Analysis

In [3]:
# Import libraries

import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re

In [6]:
# Download English module

!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.2 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
# Load spacy English module

NER = spacy.load("en_core_web_sm")

#### Q3. Load the twentieth-century text file.

In [13]:
# Load the book

with open('20th_century_Wiki.txt', 'r', errors='ignore') as file:
    data = file.read().replace('\n', '')

#### Q5. Use the text file to create a NER object.

In [16]:
book = NER(data)

In [22]:
# Visualize identified entities

displacy.render(book[273:20000], style = "ent", jupyter = True)

#### Q6. Split the sentence entities from the NER object.

In [26]:
df_sentences = [] # empty shell to store results

# Loop through sentences, get entity list for each sentence
for sent in book.sents:
    entity_list = [ent.text for ent in sent.ents]
    df_sentences.append({"sentence": sent, "entities": entity_list})
    
df_sentences = pd.DataFrame(df_sentences)

In [29]:
df_sentences.head(10)

sentence  \
0  (Key, events, of, the, 20th, century, -, Wikip...   
1                           (the, wars1.2.1Economic)   
2  (depression1.2.2The, rise, of, dictatorship1.3...   
3  (World, War, II, (, 1939–1945)1.3.1The, war, i...   
4  (days1.3.7The, war, in, the, Pacific1.3.7.1Bac...   
5  (decolonization1.4.2The, Cold, War, (, 1947–19...   
6  (The, World, Wars, sparked, tension, between, ...   
7  (These, advancements, have, played, a, signifi...   
8  (Historic, events, in, the, 20th, century[edit...   
9  (The, 1900s, saw, the, decade, herald, a, seri...   

                                            entities  
0  [the 20th century - WikipediaJump, Navigation\...  
1                                                 []  
2                                                 []  
3                                     [World War II]  
4  [Pacific1.3.7.1Background1.3.8Japanese, Holoca...  
5  [decolonization1.4.2The Cold War, 1947–1991)1....  
6  [the Cold War, the Space Race, the World Wide ...  
7                          [the 21st century, today]  
8            [the 20th, Edwardian, the 20th century]  
9                            [The 1900s, the decade]

In [33]:
# Import countries

countries_df = pd.read_csv("countries.csv", index_col = 0)

In [36]:
countries_df.head()

Country  Times Mentioned
0    France               16
1   Germany               43
2   Austria                5
3   Hungary                3
4  Bulgaria                2

#### Q7. Filter the entities so that you end up only with the ones from your countries list.

In [57]:
# Function to filter out entities not of interest

def filter_entity(ent_list, countries_df):
    return [ent for ent in ent_list 
            if ent in list(countries_df['Country'])]

In [59]:
df_sentences['countries_entities'] = df_sentences['entities'].apply(lambda x: filter_entity(x, countries_df))

In [64]:
df_sentences['countries_entities'].head(20)

0                      []
1                      []
2                      []
3                      []
4                      []
5                      []
6                      []
7                      []
8                      []
9                      []
10                     []
11                     []
12                     []
13                     []
14                     []
15                     []
16                     []
17               [France]
18    [Germany, Bulgaria]
19              [Germany]
Name: countries_entities, dtype: object

In [69]:
# Filter out sentences that don't have any country entities

df_sentences_filtered = df_sentences[df_sentences['countries_entities'].map(len) > 0]

In [72]:
df_sentences_filtered.tail(10)

sentence  \
746   (PMID,  , 21731859.^, Gutman, ,, Israel, (, 19...   
812   (", Israel, 's, Nuclear, Weapons, :, 5, Things...   
825   (Past, &, Present, ., 210, :, 103–120, ., doi:...   
846   (", The, forgotten, violence, that, helped, In...   
849   (", Indian, Independence, Day, :, everything, ...   
855   (Retrieved, 2018, -, 12, -, 18.^, ", The, Phil...   
902   (The, Moldovans, :, Romania, ,, Russia, ,, and...   
1146  (", Anti, -, American, Behavior, in, the, Midd...   
1149  (The, Rise, of, China, and, India, :, A, New, ...   
1150               (Singapore, :, World, Scientific, .)   

                                               entities countries_entities  
746                  [21731859.^, Gutman, Israel, 1990]           [Israel]  
812                                            [Israel]           [Israel]  
825   [Past & Present, 210, 103–120, Germany, The Co...          [Germany]  
846                                             [India]            [India]  
849   [Indian Independence Day, Partition, India, Pa...  [India, Pakistan]  
855   [2018-12-18.^, Philippines, 1898–1946, US Hous...      [Philippines]  
902   [Moldovans, Romania, Russia, the Politics of C...          [Romania]  
1146          [Anti-American, the Middle East, Lebanon]          [Lebanon]  
1149                    [China, India, New Asian Drama]     [China, India]  
1150                      [Singapore, World Scientific]        [Singapore]

#### Q8. Create the relationships dataframe.

In [77]:
# Defining relationships 

# window size = 5 : this defines how many sentences will be looked at simultaneously 
relationships = [] # create an empty list

for i in range(df_sentences_filtered.index[-1]):
    end_i = min(i+5, df_sentences_filtered.index[-1])
    countries_list = sum((df_sentences_filtered.loc[i: end_i].countries_entities), [])
    
    # Remove duplicated characters that are next to each other
    countries_unique = [countries_list[i] for i in range(len(countries_list)) 
                   if (i==0) or countries_list[i] != countries_list[i-1]]
    
    if len(countries_unique) > 1:
        for idx, a in enumerate(countries_unique[:-1]):
            b = countries_unique[idx + 1]
            relationships.append({"source": a, "target": b})

In [80]:
relationship_df = pd.DataFrame(relationships)

In [83]:
relationship_df

source     target
0      France    Germany
1     Germany   Bulgaria
2      France    Germany
3     Germany   Bulgaria
4    Bulgaria    Germany
..        ...        ...
562     India  Singapore
563     China      India
564     India  Singapore
565     China      India
566     India  Singapore

[567 rows x 2 columns]

In [86]:
# Sort the cases with a->b and b->a

relationship_df = pd.DataFrame(np.sort(relationship_df.values, axis = 1), columns = relationship_df.columns)
relationship_df.head(5)

source   target
0    France  Germany
1  Bulgaria  Germany
2    France  Germany
3  Bulgaria  Germany
4  Bulgaria  Germany

In [89]:
# Summarize the interactions

relationship_df["value"] = 1
relationship_df = relationship_df.groupby(["source","target"], sort=False, as_index=False).sum()

In [92]:
relationship_df.head(10)

source     target  value
0    France    Germany     36
1  Bulgaria    Germany     11
2   Germany      Italy     26
3   Austria    Germany     11
4   Germany      Spain      1
5    France     Poland     11
6   Germany     Poland     28
7   Estonia    Germany      5
8   Estonia     Latvia     12
9    Latvia  Lithuania      6

#### Q9. Save and export your dataframe.

In [95]:
relationship_df.to_csv('countries_relationship.csv')